kNN 알고리즘은 주변의 가까운 사람들에게 물어봐서 대다수늬 답변과 동일하게 답변하는 방법이다.  
만약에 3명에게 물어봤을 때 2이 거짓으로 대답을 한다면 예측은 틀리게 된다.  
서포트 벡터 머신은 '지금 현재 이곳이 한강의 북쪽인가요? 남쪽인가요?'와 같이 질문해서 답변을 얻는 방법이다.

서포트 벡터라는 개념은 한강의 위치를 찾는데 사용되는 개념으로 한강은 현재위치가 강북인지 강남인지 구분하는 결정 경계선(decision boundary)이다.  
서포트 벡터는 결정 경계선을 찾는데 사용되는 개념이고 벡터는 2차원 공간상에 나타나는 데이터 포인트를 의미한다.  
따라서, 서포트 벡터는 결정 경계선과 가장 가까이 맞닿은 데이터 포인트를 의미한다.

서포트 벡터와 결정 경계선 사이의 거리를 마진이라 하고 서포트 벡터 머신의 목표는 마진을 최대로 하는 결정 경계를 찾는 것이다.  
서포트 벡터 머신에서는 약간의 오류를 허용하기 위해 비용(C)이라는 변수를 사용한다.  
비용이 낮을수록 마진을 최대한 높이고 학습 에러율은 증가하는 방향으로 결정 경계선을 만들고,  
비용이 높을수록 마진은 낮아지고 학습 에러율은 감소하는 방향을 결정 경계를 만든다.  
비용이 너무 낮으면 과소 적합의 위험이 있고 너무 높으면 과대 적합의 위험이 있으니 적절한 비용값을 찾는 과정이 상당히 중요하다.

In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('./data/basketball_stat.csv')
df.drop(['2P', 'AST', 'STL'], axis=1, inplace=True)
train, test = train_test_split(df, test_size=0.8)
df.head()

,Player,Pos,3P,TRB,BLK
0,Alex Abrines,SG,1.4,1.3,0.1
1,Steven Adams,C,0.0,7.7,1.0
2,Alexis Ajinca,C,0.0,4.5,0.6
3,Chris Andersen,C,0.0,2.6,0.6
4,Will Barton,SG,1.5,4.3,0.5


커널 트릭(kernel trick)  
저차원 벡터 공간의 데이터를 고차원 벡터 공간으로 옮겨줌으로써 결정 경계를 찾는 방법이다.  
저처원의 데이터를 고차원의 데이터로 옮겨주는 함수를 매핑 함수라 하고 매핑 함수를 가지고 많은 양의 데이터를 저차원에서 고차원으로 옮기기에는 계산량이 너무 많아져서 현실적으로는 사용하기 어렵다.   
실제로 데이터를 고차원으로 보내지는 않지만 보낸 것과 동일한 효과를 줘서 빠른 속도를 결정 경계를 찾는 방법이 고안되었으며 이를 커널 트릭이라 한다.

최적의 SVM 파라미터 찾기  
SVM에서 RBF(가우시안) 커널을 사용하므로 마진의 너비를 조정하는 파라미터(C)와 결정 경계선의 곡률을 조정하는 파라미터(gamma)를 알아야 한다.  
사이킷런의 그리드 서치(grid search)를 개발자자 부여한 비용과 감마의 후보들을 조합해서 최적의 비용과 감마값을 알아낸다.

In [8]:
# 최적의 비용과 감마값을 알아내기 위해 GridSearchCV() 함수를 import 한다.
from sklearn.model_selection import GridSearchCV
# SVM 알고리즘을 사용하기 위해 SVC() 함수를 import 한다. SVC => Support Vector Classification
from sklearn.svm import SVC
# 분류 결과 정확도, 정밀도, 재현율, f1 점수 리포트를 출력하기 위해 classification_report() 함수를 import 한다.
from sklearn.metrics import classification_report

In [9]:
# 사이킷런에서 제공하는 GridSearchCV() 함수를 사용해서 최적의 비용과 감마값을 알아내는 함수
def svm_param_selection(x, y, nfold):
    # 비용과 감마값의 후보를 정한다.
    svm_parameters = [{
        'kernel': ['rbf'],
        'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1],
        'C': [0.01, 0.1, 1, 10, 100, 1000]
    }]
    clf = GridSearchCV(SVC(), svm_parameters, cv=nfold) # GridSearchCV 객체를 만든다.
    clf.fit(x, y.values.ravel()) # GridSearch를 학습시킨다.
    print(clf.best_score_) # 최고 점수
    print(clf.best_params_, clf.best_params_['C'], clf.best_params_['gamma']) # 최고 점수를 낸 파라미터
    print(clf.best_estimator_) # 최고 점수를 낸 파라미터를 가진 SVC객에 모델
    return clf

In [10]:
x_train = train[['3P', 'BLK']] # 3점슛과 블로킹 횟수를 학습 데이터로 지정한다.
y_train = train['Pos'] # 포지션을 실제값으로 지정한다.
clf = svm_param_selection(x_train, y_train, 10)

0.95
{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'} 100 0.1
SVC(C=100, gamma=0.1)


결정 경계 시각화

In [11]:
# 최고 점수를 낸 파라미터를 이용해서 시각화 할 비용 후보들을 저장한다.
C_canditates = []
C_canditates.append(clf.best_params_['C'] * 0.01)
C_canditates.append(clf.best_params_['C']) # 최적의 'C'값
C_canditates.append(clf.best_params_['C'] * 100)
print(C_canditates)

# 최고 점수를 낸 파라미터를 이용해서 시각화 할 감마 후보들을 저장한다.
gamma_canditates = []
gamma_canditates.append(clf.best_params_['gamma'] * 0.01)
gamma_canditates.append(clf.best_params_['gamma']) # 최적의 'gamma'값
gamma_canditates.append(clf.best_params_['gamma'] * 100)
print(gamma_canditates)

[1.0, 100, 10000]
[0.001, 0.1, 10.0]


시각화를 위해 포지션(센터: C = 0, 슈팅가드: SG = 1)을 숫자로 표현한다.

In [12]:
position = []
for i in y_train:
    if i == 'C':
        position.append(0)
    else:
        position.append(1)
print(position)

[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]


In [13]:
classifiers = []
for C in C_canditates:
    for gamma in gamma_canditates:
        clf = SVC(C=C, gamma=gamma) # 최적의 파라미터 후보들을 조합해서 서포트 벡터 머신 모델을 만든다.
        clf.fit(x_train, y_train) # 최적의 파라미터 후보들을 조합해서 서포트 벡터 머신 모델을 학습시킨다.
        classifiers.append([C, gamma, clf]) # 최적의 파라미터 후보들을 조합해서 서포트 벡터 머신 모델들을 저장한다.
classifiers

[[1.0, 0.001, SVC(gamma=0.001)],
 [1.0, 0.1, SVC(gamma=0.1)],
 [1.0, 10.0, SVC(gamma=10.0)],
 [100, 0.001, SVC(C=100, gamma=0.001)],
 [100, 0.1, SVC(C=100, gamma=0.1)],
 [100, 10.0, SVC(C=100, gamma=10.0)],
 [10000, 0.001, SVC(C=10000, gamma=0.001)],
 [10000, 0.1, SVC(C=10000, gamma=0.1)],
 [10000, 10.0, SVC(C=10000, gamma=10.0)]]

각 모델을 시각화한다.

In [ ]:
plt.figure(figsize=[20, 20])
# np.linspace(0, 4, 100): 0 ~ 4를 100단계로 나눈값을 저장하는 넘파이 배열을 만든다.
# print(np.linspace(0, 4, 100))
xx, yy = np.meshgrid(np.linspace(0, 4, 100), np.linspace(0, 4, 100)) # 그래프의 눈금

# enumerate(): 인수로 지정된 리스트를 인덱스와 값 형태의 튜플로 리턴한다.
for i, (C, gamma, clf) in enumerate(classifiers):
    # print(i, (C, gamma, clf))

    # 넘파이 배열을 옆으로, 위아래로 붙이기: np.r_(), np.c_()
    # np.r_[a, b] => a, b 두 넘파이 배열을 왼쪽에서 오른쪽으로 붙인다.
    # np.r_[[a], [b]] => a, b 두 넘파이 배열을 위에서 아래쪽으로 붙인다.
    # np.c_[a, b] => 두 개의 1차원 배열을 컬럼으로 세로로 붙여서 2차원 배열을 만든다.
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # subplot 그래프: 두 개 이상의 그래프의 상관 관계를 효과적으로 표시할 수 있다.
    # subplot을 사용하는 방법은 subplot() 함수를 사용해서 그래프의 위치를 정해주고 그래프는 scatter() 함수로 그리면 된다.
    plt.subplot(len(C_canditates), len(gamma_canditates), i + 1)
    plt.title('C = {}, gamma = {}'.format(C, gamma))
    
    # pcolormesh(): cmap으로 지정한 색상을 이용해서 xx, yy로 지정된 단계로 Z에 저장된 결정 경계면을 표시한다.
    plt.pcolormesh(xx, yy, -Z, cmap=plt.cm.RdBu)
    # x_train['3P']를 x축으로 x_train['BLK']를 y축으로하고 cmap 속성에서 지정한 색상(edgecolors는 테두리 색상)으로 c로
    # 구분해서 산점도 그래프를 그린다.
    plt.scatter(x=x_train['3P'], y=x_train['BLK'], c=position, edgecolors='k', cmap=plt.cm.RdBu_r)
plt.show()